Pulling POIS from Overpass

In [29]:
import requests
import pandas as pd
import time
import json

# ---------------- CONFIG ----------------
CITIES_CSV = "cities.csv"      # your input cities
OUTPUT_CSV = "all_cultural_pois.csv"
GEO_RATE_LIMIT = 1.0           # 1 second per geocode (Census rate limit)
OVERPASS_RATE_LIMIT = 2.0      # 2 seconds between Overpass queries
RADIUS_METERS = 10000          # 10 km radius

# ---------------- GEOCODING FUNCTION ----------------
def geocode_city(city, state):
    """
    Geocode a US city using Nominatim first, then Census as a fallback.

    Nominatim: "City, State, USA"
    Census (only if Nominatim fails): address + onelineaddress variants
    """
    # ---------- Nominatim (primary) ----------
    nominatim_url = "https://nominatim.openstreetmap.org/search"
    params_nom = {
        "q": f"{city}, {state}, USA",
        "format": "json",
        "limit": 1,
    }
    headers_nom = {
        # PUT YOUR EMAIL OR CONTACT HERE
        "User-Agent": "city-cultural-footprint/0.1 (priscavarner@yahoo.com)",
    }
    try:
        r_nom = requests.get(nominatim_url, params=params_nom,
                             headers=headers_nom, timeout=20)
        r_nom.raise_for_status()
        results = r_nom.json()
        if results:
            lat = float(results[0]["lat"])
            lon = float(results[0]["lon"])
            return lat, lon
    except Exception as e:
        print(f"  ⚠️ Nominatim error for {city}, {state}: {e}")

    # ---------- Census fallback ----------
    base_url = "https://geocoding.geo.census.gov/geocoder/locations"

    def parse_census_coords(data):
        matches = data.get("result", {}).get("addressMatches", [])
        if not matches:
            return None, None
        coords = matches[0]["coordinates"]
        return coords["y"], coords["x"]  # lat, lon

    # 1) address endpoint with dummy street
    url1 = f"{base_url}/address"
    params1 = {
        "street": "1 Main St",
        "city": city,
        "state": state,
        "benchmark": "Public_AR_Current",
        "format": "json",
    }
    try:
        r1 = requests.get(url1, params=params1, timeout=20)
        r1.raise_for_status()
        lat, lon = parse_census_coords(r1.json())
        if lat is not None:
            return lat, lon
    except Exception as e:
        print(f"  ⚠️ Census address error for {city}, {state}: {e}")

    # 2) onelineaddress: "City, ST"
    url2 = f"{base_url}/onelineaddress"
    params2 = {
        "address": f"{city}, {state}",
        "benchmark": "Public_AR_Current",
        "format": "json",
    }
    try:
        r2 = requests.get(url2, params=params2, timeout=20)
        r2.raise_for_status()
        lat, lon = parse_census_coords(r2.json())
        if lat is not None:
            return lat, lon
    except Exception as e:
        print(f"  ⚠️ Census oneline error for {city}, {state}: {e}")

    print(f"  ❌ No geocode match for {city}, {state} (Nominatim + Census)")
    return None, None

# ---------------- OVERPASS QUERY FUNCTION ----------------
def fetch_pois(lat, lon, radius_m=RADIUS_METERS, max_retries=3):
    """
    Query Overpass for cultural + entertainment POIs around a point.

    Included types:
      - Museums          (tourism=museum)
      - Galleries        (tourism=gallery)
      - Theatres         (amenity=theatre)
      - Parks            (leisure=park)
      - Stadiums         (leisure/sport/building=stadium)
      - Music venues     (amenity=music_venue)
      - Concert halls    (amenity=concert_hall)
      - Arts centres     (amenity=arts_centre)
      - Nightclubs       (amenity=nightclub)
      - Bars/pubs with live music (amenity=bar/pub + music=yes or live_music=yes)
      - Music-theatres   (amenity=theatre + theatre:genre=music)

    Logic:
      1) Try radius_m, up to max_retries.
      2) If all fail, try again with radius_m / 2.
    """

    def build_query(radius):
        return f"""
        [out:json][timeout:40];
        (
          /* --- Core cultural POIs --- */
          node["tourism"="museum"](around:{radius},{lat},{lon});
          way["tourism"="museum"](around:{radius},{lat},{lon});
          relation["tourism"="museum"](around:{radius},{lat},{lon});

          node["tourism"="gallery"](around:{radius},{lat},{lon});
          way["tourism"="gallery"](around:{radius},{lat},{lon});
          relation["tourism"="gallery"](around:{radius},{lat},{lon});

          node["amenity"="theatre"](around:{radius},{lat},{lon});
          way["amenity"="theatre"](around:{radius},{lat},{lon});
          relation["amenity"="theatre"](around:{radius},{lat},{lon});

          node["leisure"="park"](around:{radius},{lat},{lon});
          way["leisure"="park"](around:{radius},{lat},{lon});
          relation["leisure"="park"](around:{radius},{lat},{lon});

          /* --- Stadiums --- */
          node["leisure"="stadium"](around:{radius},{lat},{lon});
          way["leisure"="stadium"](around:{radius},{lat},{lon});
          relation["leisure"="stadium"](around:{radius},{lat},{lon});

          node["sport"="stadium"](around:{radius},{lat},{lon});
          way["sport"="stadium"](around:{radius},{lat},{lon});
          relation["sport"="stadium"](around:{radius},{lat},{lon});

          node["building"="stadium"](around:{radius},{lat},{lon});
          way["building"="stadium"](around:{radius},{lat},{lon});
          relation["building"="stadium"](around:{radius},{lat},{lon});

          /* --- Dedicated music venues --- */
          node["amenity"="music_venue"](around:{radius},{lat},{lon});
          way["amenity"="music_venue"](around:{radius},{lat},{lon});
          relation["amenity"="music_venue"](around:{radius},{lat},{lon});

          /* --- Concert halls / arts centres --- */
          node["amenity"="concert_hall"](around:{radius},{lat},{lon});
          way["amenity"="concert_hall"](around:{radius},{lat},{lon});
          relation["amenity"="concert_hall"](around:{radius},{lat},{lon});

          node["amenity"="arts_centre"](around:{radius},{lat},{lon});
          way["amenity"="arts_centre"](around:{radius},{lat},{lon});
          relation["amenity"="arts_centre"](around:{radius},{lat},{lon});

          /* --- Nightclubs --- */
          node["amenity"="nightclub"](around:{radius},{lat},{lon});
          way["amenity"="nightclub"](around:{radius},{lat},{lon});
          relation["amenity"="nightclub"](around:{radius},{lat},{lon});

          /* --- Bars / pubs with music tags --- */
          node["amenity"="bar"]["music"="yes"](around:{radius},{lat},{lon});
          way["amenity"="bar"]["music"="yes"](around:{radius},{lat},{lon});
          relation["amenity"="bar"]["music"="yes"](around:{radius},{lat},{lon});

          node["amenity"="bar"]["live_music"="yes"](around:{radius},{lat},{lon});
          way["amenity"="bar"]["live_music"="yes"](around:{radius},{lat},{lon});
          relation["amenity"="bar"]["live_music"="yes"](around:{radius},{lat},{lon});

          node["amenity"="pub"]["music"="yes"](around:{radius},{lat},{lon});
          way["amenity"="pub"]["music"="yes"](around:{radius},{lat},{lon});
          relation["amenity"="pub"]["music"="yes"](around:{radius},{lat},{lon});

          node["amenity"="pub"]["live_music"="yes"](around:{radius},{lat},{lon});
          way["amenity"="pub"]["live_music"="yes"](around:{radius},{lat},{lon});
          relation["amenity"="pub"]["live_music"="yes"](around:{radius},{lat},{lon});

          /* --- Theatres explicitly marked as music --- */
          node["amenity"="theatre"]["theatre:genre"="music"](around:{radius},{lat},{lon});
          way["amenity"="theatre"]["theatre:genre"="music"](around:{radius},{lat},{lon});
          relation["amenity"="theatre"]["theatre:genre"="music"](around:{radius},{lat},{lon});
        );
        out center;
        """

    url = "https://overpass-api.de/api/interpreter"

    # --- 1) Try with full radius_m, with retries ---
    query_full = build_query(radius_m)
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.post(url, data={"data": query_full}, timeout=120)
            if r.status_code >= 500 or r.status_code == 429:
                raise RuntimeError(f"Overpass HTTP {r.status_code}")
            r.raise_for_status()
            return r.json().get("elements", [])
        except Exception as e:
            print(f"  ⚠️ Overpass attempt {attempt}/{max_retries} failed: {e}")
            if attempt < max_retries:
                time.sleep(5 * attempt)  # backoff
            else:
                print("  ⚠️ All full-radius attempts failed, trying half-radius fallback")

    # --- 2) Half-radius fallback ---
    small_radius = radius_m // 2
    query_small = build_query(small_radius)
    try:
        r2 = requests.post(url, data={"data": query_small}, timeout=120)
        if r2.status_code >= 500 or r2.status_code == 429:
            raise RuntimeError(f"Overpass HTTP {r2.status_code}")
        r2.raise_for_status()
        print(f"  🌐 Half-radius fallback used ({small_radius} m)")
        return r2.json().get("elements", [])
    except Exception as e:
        print(f"  ❌ Overpass still failing at half radius: {e}")
        raise

# ---------------- MAIN PIPELINE ----------------
def main():
    print("📖 Loading cities...")
    df = pd.read_csv(CITIES_CSV)

    # Split "city and state"
    parts = df["city and state"].astype(str).str.split(",", n=1, expand=True)
    df["city"] = parts[0].str.strip()
    df["state"] = parts[1].str.strip()

    # (TEST MODE is OFF — we use all cities)
    total_cities = len(df)
    print(f"🧮 Total cities to process: {total_cities}")

    all_rows = []
    processed_cities = 0

    for idx, row in df.iterrows():
        city = row["city"]
        state = row["state"]

        print(f"\n🌎 Processing {city}, {state} ({processed_cities + 1}/{total_cities})")

        # 1) GEOCODE
        lat, lon = geocode_city(city, state)
        time.sleep(GEO_RATE_LIMIT)

        if lat is None:
            print(f"❌ Skipping {city}, {state} — no geocode")
            processed_cities += 1
            continue

        print(f"  → Coordinates: {lat}, {lon}")

        # 2) OVERPASS
        try:
            pois = fetch_pois(lat, lon)
        except Exception as e:
            print(f"❌ Overpass error for {city}, {state}: {e}")
            processed_cities += 1
            continue

        print(f"  → Found {len(pois)} POIs")

        # 3) RECORD
        for e in pois:
            if "lat" in e and "lon" in e:
                plat, plon = e["lat"], e["lon"]
            else:
                plat = e.get("center", {}).get("lat")
                plon = e.get("center", {}).get("lon")

            all_rows.append({
                "city": city,
                "state": state,
                "osm_id": e.get("id"),
                "osm_type": e.get("type"),
                "lat": plat,
                "lon": plon,
                "tags": json.dumps(e.get("tags", {}), ensure_ascii=False),
            })

        processed_cities += 1

        # ---- CHECKPOINT: every 10 cities, save current progress ----
        if processed_cities % 10 == 0:
            outdf_checkpoint = pd.DataFrame(all_rows)
            outdf_checkpoint.to_csv(OUTPUT_CSV, index=False)
            print(f"  💾 Checkpoint saved after {processed_cities}/{total_cities} cities")

        time.sleep(OVERPASS_RATE_LIMIT)

    # FINAL SAVE
    outdf = pd.DataFrame(all_rows)
    outdf.to_csv(OUTPUT_CSV, index=False)
    print("\n✅ Done! Saved:", OUTPUT_CSV)


if __name__ == "__main__":
    main()


📖 Loading cities...
🧮 Total cities to process: 1000

🌎 Processing New York, NY (1/1000)
  → Coordinates: 40.7127281, -74.0060152
  → Found 1829 POIs

🌎 Processing Los Angeles, CA (2/1000)
  → Coordinates: 34.0536909, -118.242766
  ⚠️ Overpass attempt 1/3 failed: Overpass HTTP 504
  ⚠️ Overpass attempt 2/3 failed: Overpass HTTP 504
  → Found 518 POIs

🌎 Processing Chicago, IL (3/1000)
  → Coordinates: 41.8755616, -87.6244212
  → Found 722 POIs

🌎 Processing Houston, TX (4/1000)
  → Coordinates: 29.7589382, -95.3676974
  → Found 541 POIs

🌎 Processing Phoenix, AZ (5/1000)
  → Coordinates: 33.4484367, -112.074141
  → Found 333 POIs

🌎 Processing Philadelphia, PA (6/1000)
  → Coordinates: 39.9527237, -75.1635262
  → Found 800 POIs

🌎 Processing San Antonio, TX (7/1000)
  → Coordinates: 29.4246002, -98.4951405
  → Found 290 POIs

🌎 Processing San Diego, CA (8/1000)
  → Coordinates: 32.7174202, -117.162772
  → Found 393 POIs

🌎 Processing Dallas, TX (9/1000)
  → Coordinates: 32.7762719, -96.

In [32]:
import json

df2 = pd.read_csv("all_cultural_pois.csv")

# Convert JSON-string → Python dict
df2["tags"] = df2["tags"].apply(lambda x: json.loads(x) if pd.notnull(x) else {})

In [33]:
df2

,city,state,osm_id,osm_type,lat,lon,tags
0,New York,NY,296435236,node,40.747102,-74.037955,"{'addr:city': 'Hoboken', 'addr:housenumber': '..."
1,New York,NY,357607904,node,40.706511,-74.003373,"{'ele': '1', 'gnis:feature_id': '2058381', 'he..."
2,New York,NY,357608159,node,40.712074,-74.016171,"{'ele': '0', 'gnis:feature_id': '2060108', 'le..."
3,New York,NY,357610563,node,40.713506,-73.998531,"{'addr:city': 'New York', 'addr:postcode': '10..."
4,New York,NY,357616290,node,40.777500,-73.984167,"{'ele': '21', 'gnis:feature_id': '2078977', 'l..."
...,...,...,...,...,...,...,...
197451,Lake Worth Beach,FL,1189780320,way,26.620955,-80.105660,"{'leisure': 'park', 'name': 'Lakewood Gardens ..."
197452,Lake Worth Beach,FL,1197509953,way,26.609793,-80.066057,"{'addr:city': 'Lake Worth Beach', 'addr:housen..."
197453,Lake Worth Beach,FL,1232299498,way,26.644088,-80.054700,"{'leisure': 'park', 'name': 'Spillway Park'}"
197454,Lake Worth Beach,FL,3414815,relation,26.613435,-80.046927,"{'leisure': 'park', 'name': 'Bryant Park', 'ty..."


In [37]:
def classify_poi(tags):
    """
    Create a single unified category based on all OSM tag fields.
    Priority order is deliberate (e.g., museum overrides generic building).
    """

    # Extract helpful fields
    amenity  = tags.get("amenity")
    tourism  = tags.get("tourism")
    leisure  = tags.get("leisure")
    sport    = tags.get("sport")
    building = tags.get("building")
    genre    = tags.get("theatre:genre")
    music    = tags.get("music")
    live     = tags.get("live_music")

    # --------------------------
    # 1. Cultural Core
    # --------------------------
    if tourism == "museum":
        return "museum"
    if tourism == "gallery":
        return "gallery"

    if amenity == "theatre":
        if genre == "music":
            return "music_theatre"
        return "theatre"

    if amenity in ["arts_centre", "concert_hall"]:
        return amenity

    # --------------------------
    # 2. Parks / Recreation
    # --------------------------
    if leisure == "park":
        return "park"

    # --------------------------
    # 3. Stadiums (various tagging styles)
    # --------------------------
    if leisure == "stadium" or sport == "stadium" or building == "stadium":
        return "stadium"

    # --------------------------
    # 4. Nightlife & Music Venues
    # --------------------------
    if amenity == "music_venue":
        return "music_venue"

    if amenity == "nightclub":
        return "nightclub"

    # Bars/pubs with music signals
    if amenity in ["bar", "pub"]:
        if music == "yes" or live == "yes":
            return "live_music_bar_pub"
        return amenity  # generic bar/pub

    # --------------------------
    # 5. Other cultural/tourism types
    # --------------------------
    if tourism in ["attraction", "artwork", "theme_park", "zoo"]:
        return tourism

    # --------------------------
    # 6. Fallbacks for building category
    # --------------------------
    if building in ["church", "synagogue", "mosque", "cathedral"]:
        return "religious_building"

    if building:
        return f"building:{building}"

    # --------------------------
    # 7. Default fallback
    # --------------------------
    return "other"


# Apply the classifier
df2["unified_category"] = df2["tags"].apply(classify_poi)

# Preview
df2[["city", "state", "name", "unified_category"]].head()

,city,state,name,unified_category
0,New York,NY,Monroe Center for the Arts,arts_centre
1,New York,NY,South Street Seaport Historic District,park
2,New York,NY,Pumphouse Park,park
3,New York,NY,Chatham Square,park
4,New York,NY,Seventieth Street Playground,park


water POIS

In [38]:
CITIES_CSV = "cities.csv"              # same cities file you used before
WATER_POIS_CSV = "water_pois.csv"      # detailed water features
CITY_WATER_SUMMARY_CSV = "city_water_summary.csv"

GEO_RATE_LIMIT = 1.0                   # seconds between geocoding calls
OVERPASS_RATE_LIMIT = 2.0              # seconds between Overpass queries
RADIUS_METERS = 2500                  # 10 km radius around city center


# ---------------- GEOCODING FUNCTION ----------------
def geocode_city(city, state):
    """
    Geocode a US city using Nominatim first, then Census as a fallback.
    This is similar to your existing geocode_city, but self-contained here.
    """

    # ---------- Nominatim (primary) ----------
    nominatim_url = "https://nominatim.openstreetmap.org/search"
    params_nom = {
        "q": f"{city}, {state}, USA",
        "format": "json",
        "limit": 1,
    }
    headers_nom = {
        "User-Agent": "city-water-footprint/0.1 (priscavarner@yahoo.com)",
    }
    try:
        r_nom = requests.get(
            nominatim_url,
            params=params_nom,
            headers=headers_nom,
            timeout=20,
        )
        r_nom.raise_for_status()
        results = r_nom.json()
        if results:
            lat = float(results[0]["lat"])
            lon = float(results[0]["lon"])
            return lat, lon
    except Exception as e:
        print(f"  ⚠️ Nominatim error for {city}, {state}: {e}")

    # ---------- Census fallback ----------
    base_url = "https://geocoding.geo.census.gov/geocoder/locations"

    def parse_census_coords(data):
        matches = data.get("result", {}).get("addressMatches", [])
        if not matches:
            return None, None
        coords = matches[0]["coordinates"]
        return coords["y"], coords["x"]  # lat, lon

    # 1) address endpoint with dummy street
    url1 = f"{base_url}/address"
    params1 = {
        "street": "1 Main St",
        "city": city,
        "state": state,
        "benchmark": "Public_AR_Current",
        "format": "json",
    }
    try:
        r1 = requests.get(url1, params=params1, timeout=20)
        r1.raise_for_status()
        lat, lon = parse_census_coords(r1.json())
        if lat is not None:
            return lat, lon
    except Exception as e:
        print(f"  ⚠️ Census address error for {city}, {state}: {e}")

    # 2) onelineaddress: "City, ST"
    url2 = f"{base_url}/onelineaddress"
    params2 = {
        "address": f"{city}, {state}",
        "benchmark": "Public_AR_Current",
        "format": "json",
    }
    try:
        r2 = requests.get(url2, params=params2, timeout=20)
        r2.raise_for_status()
        lat, lon = parse_census_coords(r2.json())
        if lat is not None:
            return lat, lon
    except Exception as e:
        print(f"  ⚠️ Census oneline error for {city}, {state}: {e}")

    print(f"  ❌ No geocode match for {city}, {state} (Nominatim + Census)")
    return None, None


# ---------------- OVERPASS QUERY: WATER FEATURES ----------------
def fetch_water_pois(lat, lon, radius_m=RADIUS_METERS, max_retries=3):
    """
    Query Overpass for water-related features around a point.

    Water types:
      - Lakes:       natural=water + water=lake
      - Rivers:      waterway=river/stream OR natural=water + water=river
      - Coastline:   natural=coastline
      - Beach:       natural=beach
      - Marina:      leisure=marina
      - Harbour:     any object with 'harbour' or 'harbor' tag
    """

    def build_query(radius):
        return f"""
        [out:json][timeout:40];
        (
          /* --- Lakes --- */
          node["natural"="water"]["water"="lake"](around:{radius},{lat},{lon});
          way["natural"="water"]["water"="lake"](around:{radius},{lat},{lon});
          relation["natural"="water"]["water"="lake"](around:{radius},{lat},{lon});

          /* --- Rivers / Streams --- */
          node["waterway"~"^(river|stream)$"](around:{radius},{lat},{lon});
          way["waterway"~"^(river|stream)$"](around:{radius},{lat},{lon});
          relation["waterway"~"^(river|stream)$"](around:{radius},{lat},{lon});

          node["natural"="water"]["water"="river"](around:{radius},{lat},{lon});
          way["natural"="water"]["water"="river"](around:{radius},{lat},{lon});
          relation["natural"="water"]["water"="river"](around:{radius},{lat},{lon});

          /* --- Coastline --- */
          way["natural"="coastline"](around:{radius},{lat},{lon});
          relation["natural"="coastline"](around:{radius},{lat},{lon});

          /* --- Beaches --- */
          node["natural"="beach"](around:{radius},{lat},{lon});
          way["natural"="beach"](around:{radius},{lat},{lon});
          relation["natural"="beach"](around:{radius},{lat},{lon});

          /* --- Marinas --- */
          node["leisure"="marina"](around:{radius},{lat},{lon});
          way["leisure"="marina"](around:{radius},{lat},{lon});
          relation["leisure"="marina"](around:{radius},{lat},{lon});

          /* --- Harbours / Harbors (any value) --- */
          node["harbour"](around:{radius},{lat},{lon});
          way["harbour"](around:{radius},{lat},{lon});
          relation["harbour"](around:{radius},{lat},{lon});

          node["harbor"](around:{radius},{lat},{lon});
          way["harbor"](around:{radius},{lat},{lon});
          relation["harbor"](around:{radius},{lat},{lon});
        );
        out center;
        """

    url = "https://overpass-api.de/api/interpreter"

    # --- Try full radius with retries ---
    query_full = build_query(radius_m)
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.post(url, data={"data": query_full}, timeout=120)
            if r.status_code >= 500 or r.status_code == 429:
                raise RuntimeError(f"Overpass HTTP {r.status_code}")
            r.raise_for_status()
            return r.json().get("elements", [])
        except Exception as e:
            print(f"  ⚠️ Overpass attempt {attempt}/{max_retries} failed: {e}")
            if attempt < max_retries:
                time.sleep(5 * attempt)
            else:
                print("  ⚠️ All full-radius attempts failed, trying half-radius fallback")

    # --- Half-radius fallback ---
    small_radius = radius_m // 2
    query_small = build_query(small_radius)
    try:
        r2 = requests.post(url, data={"data": query_small}, timeout=120)
        if r2.status_code >= 500 or r2.status_code == 429:
            raise RuntimeError(f"Overpass HTTP {r2.status_code}")
        r2.raise_for_status()
        print(f"  🌐 Half-radius fallback used ({small_radius} m)")
        return r2.json().get("elements", [])
    except Exception as e:
        print(f"  ❌ Overpass still failing at half radius: {e}")
        raise


# ---------------- CLASSIFY WATER TYPE ----------------
def classify_water_feature(tags):
    """
    Turn OSM water tags into a simple type label.
    """
    nat = tags.get("natural")
    water = tags.get("water")
    waterway = tags.get("waterway")
    leisure = tags.get("leisure")
    has_harbour = ("harbour" in tags) or ("harbor" in tags)

    # Lakes
    if nat == "water" and water == "lake":
        return "lake"

    # Rivers / streams
    if waterway in ["river", "stream"] or (nat == "water" and water == "river"):
        return "river"

    # Coastline
    if nat == "coastline":
        return "coastline"

    # Beach
    if nat == "beach":
        return "beach"

    # Marina
    if leisure == "marina":
        return "marina"

    # Harbour/harbor
    if has_harbour:
        return "harbor_harbour"

    # Fallback
    return "other_water"


# ---------------- MAIN PIPELINE ----------------
def main():
    print("📖 Loading cities...")
    df = pd.read_csv(CITIES_CSV)

    # Split "city and state"
    parts = df["city and state"].astype(str).str.split(",", n=1, expand=True)
    df["city"] = parts[0].str.strip()
    df["state"] = parts[1].str.strip()

    total_cities = len(df)
    print(f"🧮 Total cities to process: {total_cities}")

    all_rows = []
    processed_cities = 0

    for idx, row in df.iterrows():
        city = row["city"]
        state = row["state"]

        print(f"\n🌎 Processing {city}, {state} ({processed_cities + 1}/{total_cities})")

        # 1) GEOCODE
        lat, lon = geocode_city(city, state)
        time.sleep(GEO_RATE_LIMIT)

        if lat is None:
            print(f"❌ Skipping {city}, {state} — no geocode")
            processed_cities += 1
            continue

        print(f"  → Coordinates: {lat}, {lon}")

        # 2) OVERPASS WATER QUERY
        try:
            elements = fetch_water_pois(lat, lon)
        except Exception as e:
            print(f"❌ Overpass error for {city}, {state}: {e}")
            processed_cities += 1
            continue

        print(f"  → Found {len(elements)} water-related features")

        # 3) RECORD DETAILED ROWS
        for e in elements:
            tags = e.get("tags", {}) or {}

            # coordinates: node or center of area
            if "lat" in e and "lon" in e:
                plat, plon = e["lat"], e["lon"]
            else:
                plat = e.get("center", {}).get("lat")
                plon = e.get("center", {}).get("lon")

            all_rows.append({
                "city": city,
                "state": state,
                "osm_id": e.get("id"),
                "osm_type": e.get("type"),
                "lat": plat,
                "lon": plon,
                "water_type": classify_water_feature(tags),
                "tags": json.dumps(tags, ensure_ascii=False),
            })

        processed_cities += 1

        # Optional checkpoint every 10 cities
        if processed_cities % 10 == 0:
            tmp_df = pd.DataFrame(all_rows)
            tmp_df.to_csv(WATER_POIS_CSV, index=False)
            print(f"  💾 Checkpoint saved after {processed_cities}/{total_cities} cities")

        time.sleep(OVERPASS_RATE_LIMIT)

    # 4) FINAL SAVE: detailed water POIs
    outdf = pd.DataFrame(all_rows)
    outdf.to_csv(WATER_POIS_CSV, index=False)
    print("\n✅ Done! Saved detailed water POIs to:", WATER_POIS_CSV)

    # 5) CITY-LEVEL SUMMARY COUNTS
    if not outdf.empty:
        summary = (
            outdf.groupby(["city", "state", "water_type"])
                 .size()
                 .reset_index(name="count")
        )
        summary.to_csv(CITY_WATER_SUMMARY_CSV, index=False)
        print("✅ Saved city water summary to:", CITY_WATER_SUMMARY_CSV)
    else:
        print("⚠️ No water features found; summary not created.")


if __name__ == "__main__":
    main()

📖 Loading cities...
🧮 Total cities to process: 1000

🌎 Processing New York, NY (1/1000)
  → Coordinates: 40.7127281, -74.0060152
  → Found 38 water-related features

🌎 Processing Los Angeles, CA (2/1000)
  → Coordinates: 34.0536909, -118.242766
  → Found 3 water-related features

🌎 Processing Chicago, IL (3/1000)
  → Coordinates: 41.8755616, -87.6244212
  → Found 28 water-related features

🌎 Processing Houston, TX (4/1000)
  → Coordinates: 29.7589382, -95.3676974
  → Found 14 water-related features

🌎 Processing Phoenix, AZ (5/1000)
  → Coordinates: 33.4484367, -112.074141
  ⚠️ Overpass attempt 1/3 failed: Overpass HTTP 429
  ⚠️ Overpass attempt 2/3 failed: Overpass HTTP 429
  ⚠️ Overpass attempt 3/3 failed: Overpass HTTP 504
  ⚠️ All full-radius attempts failed, trying half-radius fallback
  🌐 Half-radius fallback used (1250 m)
  → Found 0 water-related features

🌎 Processing Philadelphia, PA (6/1000)
  → Coordinates: 39.9527237, -75.1635262
  → Found 47 water-related features

🌎 Proc

merge of all vibe columns

In [39]:
import pandas as pd
import json

# ---------------- CONFIG ----------------
CULTURAL_CSV = "all_cultural_pois.csv"       # from your cultural venues pipeline
WATER_SUMMARY_CSV = "city_water_summary.csv" # from the water_pois script
OUTPUT_MERGED_CSV = "city_vibrancy_merged.csv"


# ---------------- CLASSIFIER (same as before) ----------------
def classify_poi(tags):
    """
    Create a single unified category based on OSM tag fields.
    """

    amenity  = tags.get("amenity")
    tourism  = tags.get("tourism")
    leisure  = tags.get("leisure")
    sport    = tags.get("sport")
    building = tags.get("building")
    genre    = tags.get("theatre:genre")
    music    = tags.get("music")
    live     = tags.get("live_music")

    # 1. Cultural core
    if tourism == "museum":
        return "museum"
    if tourism == "gallery":
        return "gallery"

    if amenity == "theatre":
        if genre == "music":
            return "music_theatre"
        return "theatre"

    if amenity in ["arts_centre", "concert_hall"]:
        return amenity

    # 2. Parks / recreation
    if leisure == "park":
        return "park"

    # 3. Stadiums
    if leisure == "stadium" or sport == "stadium" or building == "stadium":
        return "stadium"

    # 4. Nightlife & music
    if amenity == "music_venue":
        return "music_venue"

    if amenity == "nightclub":
        return "nightclub"

    if amenity in ["bar", "pub"]:
        if music == "yes" or live == "yes":
            return "live_music_bar_pub"
        return amenity  # generic bar/pub

    # 5. Other tourism
    if tourism in ["attraction", "artwork", "theme_park", "zoo"]:
        return tourism

    # 6. Buildings
    if building in ["church", "synagogue", "mosque", "cathedral"]:
        return "religious_building"

    if building:
        return f"building:{building}"

    # 7. Fallback
    return "other"


# ---------------- STEP 1: CULTURAL SUMMARY ----------------
print("📥 Loading cultural POIs...")
df2 = pd.read_csv(CULTURAL_CSV)

# Parse JSON tags
df2["tags"] = df2["tags"].apply(
    lambda x: json.loads(x) if isinstance(x, str) and x.strip() != "" else {}
)

# Unified category
df2["unified_category"] = df2["tags"].apply(classify_poi)

# Pivot: counts of each category per city
print("🔢 Building cultural category pivot per city...")
cultural_pivot = (
    df2.pivot_table(
        index=["city", "state"],
        columns="unified_category",
        values="osm_id",
        aggfunc="count",
        fill_value=0,
    )
    .reset_index()
)

# Optional: add a total cultural venues column
cultural_pivot["total_cultural_pois"] = (
    cultural_pivot.drop(columns=["city", "state"]).sum(axis=1)
)

print("✅ Cultural pivot shape:", cultural_pivot.shape)


# ---------------- STEP 2: WATER SUMMARY ----------------
print("📥 Loading water summary...")
water_summary = pd.read_csv(WATER_SUMMARY_CSV)

# Pivot: counts of each water_type per city
print("🔢 Building water type pivot per city...")
water_pivot = (
    water_summary.pivot_table(
        index=["city", "state"],
        columns="water_type",
        values="count",
        aggfunc="sum",
        fill_value=0,
    )
    .reset_index()
)

# Optional: add total water features per city
water_type_cols = [c for c in water_pivot.columns if c not in ["city", "state"]]
water_pivot["total_water_features"] = water_pivot[water_type_cols].sum(axis=1)

print("✅ Water pivot shape:", water_pivot.shape)


# ---------------- STEP 3: MERGE CULTURAL + WATER ----------------
print("🔗 Merging cultural and water summaries per city...")
merged = pd.merge(
    cultural_pivot,
    water_pivot,
    on=["city", "state"],
    how="left",  # keep all cities, even if no water features
)

# Fill NaNs from merge with 0 for water-related columns
for col in merged.columns:
    if col not in ["city", "state"] and merged[col].dtype != "O":
        merged[col] = merged[col].fillna(0)

print("✅ Merged shape:", merged.shape)

# ---------------- STEP 4: SAVE ----------------
merged.to_csv(OUTPUT_MERGED_CSV, index=False)
print("💾 Saved merged city-level summary to:", OUTPUT_MERGED_CSV)


📥 Loading cultural POIs...
🔢 Building cultural category pivot per city...
✅ Cultural pivot shape: (995, 14)
📥 Loading water summary...
🔢 Building water type pivot per city...
✅ Water pivot shape: (947, 9)
🔗 Merging cultural and water summaries per city...
✅ Merged shape: (995, 21)
💾 Saved merged city-level summary to: city_vibrancy_merged.csv


ticket master API for events, only ended up using venues no event data

In [5]:
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

# -----------------------------
# CONFIG
# -----------------------------
API_KEY = "QOBJXr4wdg2Cy8wa6ENLLwOwsJGTLvoZ"   # <-- your Ticketmaster key
INPUT_CSV = "cities.csv"
EVENT_OUTPUT_CSV = "ticketmaster_events_next90d_bysegment.csv"
SUMMARY_OUTPUT_CSV = "ticketmaster_city_summary_next90d_bysegment.csv"

# Segments we’ll query separately
SEGMENTS = [
    "Music",
    "Arts & Theatre",
    "Sports",
]

# -----------------------------
# HELPER: CLASSIFY VENUE TYPE
# -----------------------------
def classify_venue_type(venue_name):
    """
    Infer venue type from name patterns.
    Returns one of:
    stadium_arena, amphitheatre, concert_hall, theatre, club_bar,
    ballroom_hall, casino, park, pavilion, fairground, other
    """
    if not venue_name:
        return "other"

    name = venue_name.lower()

    # Stadium / arena / sports
    if any(x in name for x in ["stadium", "arena", "ballpark", "field", "center", "centre", "coliseum", "dome"]):
        return "stadium_arena"

    # Amphitheater
    if "amphitheater" in name or "amphitheatre" in name:
        return "amphitheatre"

    # Concert halls / symphony
    if any(x in name for x in ["concert hall", "symphony", "orchestra", "philharmonic"]):
        return "concert_hall"

    # Theatres / performing arts
    if any(x in name for x in ["theatre", "theater", "playhouse", "opera", "performing arts",
                               "arts center", "arts centre"]):
        return "theatre"

    # Clubs / bars / lounges
    if any(x in name for x in ["club", "lounge", "bar", "pub", "saloon", "tavern", "taproom"]):
        return "club_bar"

    # Ballroom / hall
    if any(x in name for x in ["ballroom", "hall"]):
        return "ballroom_hall"

    # Casinos
    if "casino" in name:
        return "casino"

    # Parks / pavilions / fairgrounds
    if "fairground" in name or "fairgrounds" in name:
        return "fairground"
    if "pavilion" in name:
        return "pavilion"
    if "park" in name:
        return "park"

    return "other"


# -----------------------------
# NORMALIZE SEGMENT LABEL
# -----------------------------
def normalize_segment_label(segment_name):
    """
    Turn Ticketmaster segment label into a canonical short label.
    We’ll use this to build columns like music_events, sports_venues, etc.
    """
    if not segment_name:
        return "other"

    s = segment_name.lower()
    if "music" in s:
        return "music"
    if "arts" in s or "theatre" in s or "theater" in s:
        return "arts_theatre"
    if "sport" in s:
        return "sports"
    return "other"


# ---------------------------------
# GET EVENTS FOR ONE SEGMENT
# ---------------------------------
def get_events_for_segment(city, state, segment_label, api_key,
                           days_forward=90, max_pages=10):
    """
    Fetch Ticketmaster events for a city/state for the NEXT `days_forward` days,
    filtered by classificationName = segment_label (e.g., 'Music').

    Returns a list of raw event JSON objects.
    """
    url = "https://app.ticketmaster.com/discovery/v2/events"

    now = datetime.utcnow()
    end = now + timedelta(days=days_forward)

    params = {
        "apikey": api_key,
        "city": city,
        "stateCode": state,
        "countryCode": "US",
        "classificationName": segment_label,   # key part: Music / Arts & Theatre / Sports
        "size": 200,
        "sort": "date,asc",
        "startDateTime": now.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "endDateTime":   end.strftime("%Y-%m-%dT%H:%M:%SZ"),
    }

    events = []
    page = 0

    print(f"   → Segment: {segment_label}")

    while True:
        if page >= max_pages:
            print(f"     ⚠️ Reached max_pages={max_pages} for {city}, {state}, {segment_label}; stopping pagination")
            break

        params["page"] = page

        try:
            r = requests.get(url, params=params, timeout=20)
        except Exception as e:
            print(f"     ⚠️ Request error on page {page} for {city}, {state}, {segment_label}: {e}")
            break

        if r.status_code == 400:
            # Often means invalid page or some filter issue; keep what we have so far
            print(f"     ⚠️ 400 Bad Request on page {page} for {city}, {state}, {segment_label}; stopping this segment")
            break

        r.raise_for_status()
        data = r.json()

        if "_embedded" not in data or "events" not in data["_embedded"]:
            break

        batch = data["_embedded"]["events"]
        events.extend(batch)

        page_info = data.get("page", {})
        total_pages = page_info.get("totalPages", 1)

        if page >= total_pages - 1:
            break

        page += 1

    print(f"   → {segment_label}: fetched {len(events)} events")
    return events


# -------------------------
# MAIN PIPELINE
# -------------------------
def main(test_mode=False, days_forward=90):
    df = pd.read_csv(INPUT_CSV)

    # Split "City, ST"
    parts = df["city and state"].astype(str).str.split(",", n=1, expand=True)
    df["city"] = parts[0].str.strip()
    df["state"] = parts[1].str.strip()

    # TEST MODE: only first 1 city
    if test_mode:
        df = df.head(1)
        print("🧪 TEST MODE: running only first 1 city")
    else:
        print(f"🧮 Running all {len(df)} cities")

    all_rows = []

    for idx, row in df.iterrows():
        city = row["city"]
        state = row["state"]

        print(f"\n🎫 Pulling NEXT {days_forward} days of events for {city}, {state}")

        for seg in SEGMENTS:
            # Pull events for this segment
            try:
                raw_events = get_events_for_segment(
                    city=city,
                    state=state,
                    segment_label=seg,
                    api_key=API_KEY,
                    days_forward=days_forward,
                    max_pages=10,
                )
            except Exception as e:
                print(f"❌ Error for {city}, {state}, segment={seg}: {e}")
                continue

            # Flatten and classify
            for e in raw_events:
                embedded = e.get("_embedded", {}) or {}
                venues = embedded.get("venues", [{}])
                venue = venues[0] if venues else {}

                # Ticketmaster classification block (may be missing)
                cls = (e.get("classifications") or [{}])[0]
                tm_segment = (cls.get("segment") or {}).get("name")
                tm_genre = (cls.get("genre") or {}).get("name")
                tm_subgenre = (cls.get("subGenre") or {}).get("name")

                # Our normalized event type from TM segment name
                norm_event_type = normalize_segment_label(tm_segment or seg)
                venue_type = classify_venue_type(venue.get("name"))

                all_rows.append({
                    "city": city,
                    "state": state,
                    "segment_query": seg,                 # 'Music', 'Arts & Theatre', 'Sports'
                    "event_type": norm_event_type,        # 'music', 'arts_theatre', 'sports', 'other'
                    "event_id": e.get("id"),
                    "event_name": e.get("name"),
                    "tm_segment": tm_segment,
                    "tm_genre": tm_genre,
                    "tm_subgenre": tm_subgenre,
                    "date_local": e.get("dates", {}).get("start", {}).get("localDate"),
                    "time_local": e.get("dates", {}).get("start", {}).get("localTime"),
                    "date_utc": e.get("dates", {}).get("start", {}).get("dateTime"),
                    "venue_id": venue.get("id"),
                    "venue_name": venue.get("name"),
                    "venue_type": venue_type,
                    "venue_city": (venue.get("city") or {}).get("name"),
                    "venue_state": (venue.get("state") or {}).get("stateCode"),
                    "venue_country": (venue.get("country") or {}).get("countryCode"),
                    "venue_lat": (venue.get("location") or {}).get("latitude"),
                    "venue_lon": (venue.get("location") or {}).get("longitude"),
                })

            # Small sleep per segment per city to be nice to the API
            time.sleep(0.5)

    # -------------------------
    # SAVE EVENT-LEVEL DATA
    # -------------------------
    events_df = pd.DataFrame(all_rows)
    events_df.to_csv(EVENT_OUTPUT_CSV, index=False)
    print(f"\n💾 Saved event-level data to {EVENT_OUTPUT_CSV}")

    if events_df.empty:
        print("⚠️ No events found, skipping summary.")
        return

    # -------------------------
    # BUILD CITY-LEVEL SUMMARY
    # -------------------------
    # We'll aggregate by our normalized event_type
    agg = (
        events_df
        .groupby(["city", "state", "event_type"])
        .agg(
            events_count=("event_id", "nunique"),
            venues_count=("venue_id", "nunique"),
        )
        .reset_index()
    )

    # Pivot events
    events_wide = agg.pivot_table(
        index=["city", "state"],
        columns="event_type",
        values="events_count",
        fill_value=0,
    )
    events_wide.columns = [f"{c}_events" for c in events_wide.columns]

    # Pivot venues
    venues_wide = agg.pivot_table(
        index=["city", "state"],
        columns="event_type",
        values="venues_count",
        fill_value=0,
    )
    venues_wide.columns = [f"{c}_venues" for c in venues_wide.columns]

    summary = pd.concat([events_wide, venues_wide], axis=1).reset_index()
    summary.to_csv(SUMMARY_OUTPUT_CSV, index=False)
    print(f"💾 Saved city-level summary to {SUMMARY_OUTPUT_CSV}")


# -------------------------
# ENTRY POINT
# -------------------------
if __name__ == "__main__":
    # First run in test_mode=True to verify everything for 1 city
    #main(test_mode=True, days_forward=90)

    # When you're happy with the output, run the full thing by calling:
    main(test_mode=False, days_forward=90)

🧮 Running all 1000 cities

🎫 Pulling NEXT 90 days of events for New York, NY
   → Segment: Music
     ⚠️ 400 Bad Request on page 5 for New York, NY, Music; stopping this segment
   → Music: fetched 1000 events
   → Segment: Arts & Theatre
     ⚠️ 400 Bad Request on page 5 for New York, NY, Arts & Theatre; stopping this segment
   → Arts & Theatre: fetched 1000 events
   → Segment: Sports
   → Sports: fetched 89 events

🎫 Pulling NEXT 90 days of events for Los Angeles, CA
   → Segment: Music
   → Music: fetched 522 events
   → Segment: Arts & Theatre
   → Arts & Theatre: fetched 176 events
   → Segment: Sports
   → Sports: fetched 77 events

🎫 Pulling NEXT 90 days of events for Chicago, IL
   → Segment: Music
   → Music: fetched 470 events
   → Segment: Arts & Theatre
   → Arts & Theatre: fetched 479 events
   → Segment: Sports
   → Sports: fetched 123 events

🎫 Pulling NEXT 90 days of events for Houston, TX
   → Segment: Music
   → Music: fetched 182 events
   → Segment: Arts & Theatre